In [1]:
# ! pip install -r requirements.txt

In [1]:
API_URL="http://116.202.111.229:8000"
API_KEY="Ir7nzlSO3ZBEdGV59Ll0FMVtPsNd85Cz"
DATA_PATH = '/root/HBN/Tournament/DS/'
CACHE_DIR = '/workspace/octavf'

# TOURNAMENT

In [2]:
from evaluator import Evaluator
from api_client import APIClient

ev = Evaluator(api_url=API_URL, api_key=API_KEY, api_client=APIClient(API_URL, API_KEY),
               DATA_PATH=DATA_PATH, cache_dir=CACHE_DIR)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# ev.api_client.get_next_hint_for_current_company()
# ev.api_client.reset_current_context()

In [4]:
ev.evaluate(sleep=True)

200 {'company_id': 149, 'level': 1, 'hint': 'Black Diamond Pumping'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/root/.env/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Answered {'answer': 'Building Material and Garden Equipment and Supplies Dealers'}
Verdict 200 {'result': 'wrong', 'score': -50.0, 'answer': '562 Waste Management and Remediation Services'}
200 {'company_id': 149, 'level': 2, 'hint': 'Septic System Installs | Septic Inspection | Waste Water Treatment | Service Provider | Fully Licensed and Insured | Maintenance Services'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Waste Management and Remediation Services'}
Verdict 200 {'result': 'correct', 'score': 400.0, 'answer': '562 Waste Management and Remediation Services'}
200 {'company_id': 149, 'level': 3, 'hint': 'Black Diamond Pumping specializes in Colorado Front Range septic pumping, installation and inspection. We serve Brighton, Fort Lupton and the surrounding areas.'}


...

Answered {'answer': 'Waste Management and Remediation Services'}
Verdict 200 {'result': 'correct', 'score': 300.0, 'answer': '562 Waste Management and Remediation Services'}
200 {'company_id': 149, 'level': 4, 'hint': 'Black Diamond Pumping is a family-owned septic company based in Colorado that specializes in septic pumping, maintenance, installation, and inspection services for both residential and commercial customers in Brighton, Fort Lupton, and the surrounding areas. With over 20 years of industry experience, they offer a complete range of septic system services, including required pumping, general maintenance, waste water treatment, and inspections. They are fully equipped to handle tanks of all sizes and can service any septic tank along the front range of Northern Colorado. Black Diamond Pumpling is known for their fast, clean, and reliable services and guarantees to meet and exceed customer expectations.'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Repair and Maintenance'}
Verdict 200 {'result': 'wrong', 'score': -200.0, 'answer': '562 Waste Management and Remediation Services'}
200 {'company_id': 149, 'level': 5, 'hint': 'Garbage Collection & Waste Disposal'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Waste Management and Remediation Services'}
Verdict 200 {'result': 'correct', 'score': 100.0, 'answer': '562 Waste Management and Remediation Services'}
200 {'response': 'Wait for 99.76085495948792 seconds'}


# CHALLENGE

In [3]:
from classifier import NAICSClassifier, TokenizerWrapper
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
from collections import defaultdict

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True, cache_dir=CACHE_DIR)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left", cache_dir=CACHE_DIR)
tokenizer = TokenizerWrapper(tokenizer, model)
n2 = NAICSClassifier(tokenizer=tokenizer, naics_df=f'/{DATA_PATH}/ml_tournament/resources/NAICS2_324254.csv')
naics2_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS2_324254.csv")
naics4_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS4_324254.csv")
naics6_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS6_324254.csv")

company_hints = []
response = defaultdict(str)

n2_classes_scores = n2.classify_list_of_hints(company_hints)
naics_code = naics2_df[naics2_df['naics_label'] == n2.naics3_choices[np.argmax(n2_classes_scores)]]['naics_code']
response['naics2'] = naics_code

# based on the result, n4:
n4 = NAICSClassifier(tokenizer=tokenizer, 
                     naics_df=naics4_df[naics4_df['naics_code'].apply(lambda subnsc: subnsc.startswith(naics_code))])
n4_classes_scores = n4.classify_list_of_hints(company_hints)
naics_code = naics4_df[naics4_df['naics_label'] == n4.naics3_choices[np.argmax(n4_classes_scores)]]['naics_code']
response['naics4'] = naics_code

# based on the result, n6:
n6 = NAICSClassifier(tokenizer=tokenizer, 
                     naics_df=naics6_df[naics6_df['naics_code'].apply(lambda subnsc: subnsc.startswith(naics_code))])

n6_classes_scores = n6.classify_list_of_hints(company_hints)
naics_code = naics6_df[naics6_df['naics_label'] == n6.naics3_choices[np.argmax(n6_classes_scores)]]['naics_code']
response['naics6'] = naics_code

print(response)


NameError: name 'CACHE_DIR' is not defined